In [2]:
%load_ext autoreload
%autoreload 2

# RPN (Reverse Polish Notation) Calculator
## Team Members:
* Maziar Bastani
* Cody Broxton
* Daniel Henderson

User inputs a string of single-char variables and arithmetic operators, terminated by a dollar sign.

```ab+cd+*$```

Next, user is prompted to supply values for each variable.

Finally, expression is evaluated using postfix rules of evaluation, and the answer is printed.

In [3]:
#!/usr/bin/env python3
#
# Team Members:
# Maziar Bastani
# Cody Broxton
# Daniel Henderson

import operator

def setup():
    global tokens
    tokens = {}
    tokens.clear()
    global token_counter
    token_counter = 1
    global user_string
    user_string = ''
    global value_stack
    value_stack = []

def tokenize(string):
    operands = 0
    if len(string) == 0:
        return 'done'
    for s in string:
        if s.isalpha():
            add_token(s)
            operands += 1
        elif s in ('+', '-', '*', '/'):
            # binary operators take two operands
            # see if there are enough operands
            if operands < 2:
                print("error out. not enough operands")
                return('error')
            else:
                # remove operand, since result will be pushed as new operand
                operands -= 1
        elif s in ('$'):
            return 'done'
        elif s.isspace():
            pass
        else:
            print('illegal character reached')
            return('error')
            
def add_token(character):
    global token_counter, tokens
    if character not in tokens:
        tokens[character] = [token_counter, 0]
        token_counter += 1
        
def user_input_pass():
    global show_debug
    global tokens,  user_string
    user_string = input('Enter a postscript expression with a $ at the end: ')
    if tokenize(user_string) == 'done':
        if show_debug:
            print(tokens)
            print('tokenized successfully')
    else:
        print('tokenizer error')
    # assign_values()
        
def assign_values():
    global show_debug
    global tokens
    sorted_tokens = sorted(tokens.items(), key=operator.itemgetter(1))
    for variable in sorted_tokens:
        keyval = variable[0]
        varval = input('Enter value of ' + keyval + ': ')
        # should error out if can't convert to a float
        tokens[keyval][1] = float(varval)
    if show_debug:
        print("Token dictionary:")
        print(tokens)
        print("Sorted tokens:")
        print(sorted_tokens)
    
def compute():
    global tokens, user_string, value_stack
    for char in user_string:
        if char.isalpha():
            value_stack.append(tokens[char][1])
        if char in ('+', '-', '*', '/'):
            # binary operators
            # check that stack can handle
            if len(value_stack) > 1:
                B = value_stack.pop()
                A = value_stack.pop()
                if char == '+':
                    value_stack.append(A + B)
                elif char == '-':
                    value_stack.append(A - B)
                elif char == '*':
                    value_stack.append(A * B)
                elif char == '/':
                    if B == 0:
                        print('error. Divide by zero.')
                        value_stack.append(float('NaN'))
                        return('error')
                    value_stack.append(A / B)
                elif char.isspace():
                    pass
                else:
                    print("Unimplemented binary operator. Calculations will be wrong")
                    value_stack.append(A)
            else:
                print('Error: Stack underflow.')
                return('error')
        
def show_result():
    global show_debug
    global value_stack, tokens, value_stack
    if show_debug:
        print(tokens)
        print(value_stack)
    print("\nFinal Value = " + str(value_stack.pop()))
    
def run(debug=False):
    global show_debug
    show_debug = debug
    
    # Python equivalent of do-while loop
    while True:
        setup()    
        user_input_pass()
        assign_values()
        compute()
        show_result()
        if input("Continue(y/n)? ") != 'y':
            break

def main():
    if input("run in debug mode? ") != 'y':
        run()
    else:
        run(True)

if __name__ == '__main__':
    main()

run in debug mode? n
Enter a postscript expression with a $ at the end: lb+$
Enter value of l: 4
Enter value of b: 5

Final Value = 9.0
Continue(y/n)? n


In [5]:
run()

Enter a postscript expression with a $ at the end: ab*ac+*$
Enter value of a: 2
Enter value of b: 3
Enter value of c: 4

Final Value = 36.0
Continue(y/n)? y
Enter a postscript expression with a $ at the end: beef*++$
Enter value of b: 2
Enter value of e: 3
Enter value of f: 4

Final Value = 17.0
Continue(y/n)? n


The code in the second Cell is a proper python3 program. If that cell is selected, the code can be run with **`<shift>+<enter>`** on the keyboard. This will do the pythonic step of checking if a top-level object called **`__main__`** is defined. `__main__` is defined if the code is started from this file, but it isn't if this file was loaded as an include in another file (actually, it's defined for that file, but not this one). 

There may be a more jupyteric way to do this, but as-is, the code also runs properly when copied and pasted into a file and run from the command line.

The other way you can run it, as seen in the cell above, is to simply call the **`run()`** function in any new Cell, as seen in [5]. This method would require that the code has been run once from the first Cell, so that all the functions and methods have been loaded into the python interpreter. This technique may become more useful as the compiler project grows, since this is a means of quickly and easily testing sections of code.

Along these lines, variables from the run can be inspected in a code Cell as well. Below, we inspect the **`tokens`** dictionary by just typing in it's name. Its value is then reported.

In [8]:
tokens

{'a': [1, 3.0], 'b': [2, 4.0]}

An improvement on this code would be to have the `tokens` datastructure be a dict of dicts. 

The first element of the array holds the order in which variables were created while being parsed from the user's RPN equation. It's used to insure that the user is prompted for the values of these variables in the same order his equation has them. Since a dict can't guarantee the order over which its items are iterated, this little bit of information is the data that allows us to do this in a user-friendly, less confusing way. 

Unfortunately, at this time, I don't know how to use a value from within a dict as the means of creating a sorted structure. The current code is this:

```sorted_tokens = sorted(tokens.items(), key=operator.itemgetter(1))```

The `itemgetter()` part, I believe, is retrieving the first item in the array, the counter indicating what order tokens were created. There must be a way to get at a value, based on a key, but I don't understand the `operator` package and what it does. This part of code was cargo-culted on from a stackoverflow.com search result.